In [71]:
%matplotlib inline

In [207]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [208]:
# Unicode string to ASCII
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s) #NFD, Normalization Form Decomposition 標準分解
        if unicodedata.category(c) != 'Mn'
    )


## unicodedata.normalize講解

In [75]:
###unicodedata.normalize
s1 = 'Spicy Jalape\u00f1o' #整体字符
s2 = 'Spicy Jalapen\u0303o' #原始字母＋組合字符
s1 == s2
#len(s1)
#len(s2)

False

In [76]:
s2

'Spicy Jalapeño'

In [77]:
s1

'Spicy Jalapeño'

In [78]:
t3 = unicodedata.normalize('NFD', s1)
t4 = unicodedata.normalize('NFD', s2)
t3 == t4

True

In [46]:
# 刪除不必要的小寫、刪除非字母字符

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


## unicodedata.normalize講解結束

In [1]:
#要讀取數據文件，我們將文件拆分為行
#這些文件都是英文→其他語言，所以如果我們
#想翻譯其他語言→英語添加了``reverse``標誌反轉


In [209]:
def readLangs(lang1, lang2, reverse=False): #lang代表某個語言
    print("Reading lines...")

    #已經於Name Classification說明過
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances #將左邊與右邊資料互轉
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [48]:
#lines = open('data/%s-%s.txt' % ("eng", "fra"), encoding='utf-8').read().strip().split('\n')

In [49]:
pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

In [25]:
#字元處理
unicodeToAscii(lines[0].lower().strip())

'go.\tva !'

In [29]:
s = unicodeToAscii(lines[0].lower().strip())

In [30]:
s

'go.\tva !'

In [28]:
re.sub(r"([.!?])", r" \1", s)

'go .\tva  !'

In [31]:
re.sub(r"[^a-zA-Z.!?]+", r" ", s)

'go. va !'

In [33]:
#由於範例句子很多，我們想訓練快一點，因此將數據修剪較短的句子，也就是將一句話的單詞長度為10(包括標點符號)
#尤其像是有eng_prefixes得相關的，“我是”或“他是”等。

In [210]:
#做中文這一段就考量是否要加快速度
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p): #其中p的0與1代表 eng與fra的翻譯 #透過空白分開，確認該句是否長度小於MAX_LENGTH
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH
    
#def filterPair(p): #其中p的0與1代表 eng與fra的翻譯 #透過空白分開，確認該句是否長度小於MAX_LENGTH
#    return len(p[0].split(' ')) < MAX_LENGTH and \
#        len(p[1].split(' ')) < MAX_LENGTH and \
#        p[1].startswith(eng_prefixes)
        
        

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [122]:
pairs[0][0]

'go .'

In [121]:
#input_lang, output_lang, pairs = readLangs("eng", "fra", reverse=False)
pairs[0][1].split(' ')

['va', '!']

In [159]:
len(pairs[64137][1].split(' '))< MAX_LENGTH and \
len(pairs[64137][1].split(' '))< MAX_LENGTH

True

In [211]:
SOS_token = 0 #開始
EOS_token = 1 #結束


class Lang: #做一個Class為Lang，方便等下進行整合
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [82]:
#準備數據的完整過程是：

# - 讀取文本文件並拆分成行，將行拆分成對
# - 標準化文本，按長度和內容過濾
# - 從成對的句子中製作單詞列表

In [212]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs) #過濾較長的句子
    print("Trimmed to %s sentence pairs" % len(pairs)) #顯示修剪的句子
    print("Counting words...")
    for pair in pairs: 
        input_lang.addSentence(pair[0]) #每一個組句子印出取得，同時塞到word2index
        output_lang.addSentence(pair[1])#每一個組句子印出取得，同時塞到word2index
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))


Reading lines...
Read 135842 sentence pairs
Trimmed to 95170 sentence pairs
Counting words...
Counted words:
fra 16813
eng 10025
['quelle peut etre la signification de ceci ?', 'what can be the meaning of this ?']


In [84]:
print(random.choice(pairs))

['we eat more processed food than natural food .', 'nous mangeons davantage de nourriture industrielle que de naturelle .']


In [163]:
input_lang.addSentence(pairs[0][0])

In [165]:
for word in sentence.split(' '):
    self.addWord(word)

<bound method Lang.addSentence of <__main__.Lang object at 0x112c903c8>>

In [191]:
for word in pairs[0][0].split(' '):
    print (word)

va
!


The Encoder
-----------

seq2seq網絡的編碼器是輸出某個值的RNN
輸入句子中的每個單詞。對於編碼器的每個輸入字
輸出向量和隱藏狀態，並使用隱藏狀態
下一個輸入詞。





In [213]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)  #GRU效能比LSTM更好，GRU只剩下Update Gate

    def forward(self, input, hidden): #主要是以這個為網路跑的主線
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self): #初始化隱藏層
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [177]:
##一個簡單的查找表，用於存儲固定字典和大小的嵌入。
#此模塊通常用於存儲單詞嵌入並使用索引檢索它們。模塊的輸入是索引列表，輸出是相應的字嵌入。

In [87]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(10, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
embedding(input)

tensor([[[-0.8184, -1.1779, -0.0330],
         [-0.1729,  1.6461, -1.0431],
         [ 2.1603, -0.9625,  0.6453],
         [-0.0323,  0.4391, -1.8454]],

        [[ 2.1603, -0.9625,  0.6453],
         [-1.5644,  0.5266,  0.7882],
         [-0.1729,  1.6461, -1.0431],
         [ 0.7452,  1.1130,  1.5820]]])

In [86]:
# example with padding_idx
embedding = nn.Embedding(10, 3, padding_idx=0)
input = torch.LongTensor([[0,2,0,5]])
embedding(input)

tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 0.9413, -0.3064,  0.1697],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.2806,  0.1690,  0.9520]]])

The Decoder
-----------

解碼器是另一個獲取編碼器輸出矢量和的RNN輸出一系列單詞以創建翻譯。


Simple Decoder
^^^^^^^^^^^^^^

在最簡單的seq2seq解碼器中，我們只使用編碼器的最後一個輸出。
最後一個輸出有時在編碼時稱為*上下文向量*
整個序列的上下文。該上下文向量用作
解碼器的初始隱藏狀態。

在解碼的每個步驟中，給予解碼器輸入令牌和
隱藏的狀態。初始輸入標記是字符串開頭``<SOS>``
令牌，第一個隱藏狀態是上下文向量（編碼器的
最後隱藏的狀態）。






In [214]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size) #線性轉換
        self.softmax = nn.LogSoftmax(dim=1) #透過logSoftmax做歸一化，加上log的softmax，logsoftmax比較有效率

    def forward(self, input, hidden): #主要是以為網路跑的主線
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output) #乘以Relu
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

我鼓勵你訓練和觀察這個模型的結果，但是
節省時間，我們將直接尋找黃金並引入
注意機制。



Attention Decoder
^^^^^^^^^^^^^^^^^

如果僅在編碼器和解碼器之間傳遞上下文向量，
單個向量承擔編碼整個句子的負擔。

注意允許解碼器網絡“聚焦”在不同的部分
解碼器自身輸出的每一步的編碼器輸出。第一
我們計算一組*注意力量*。這些將乘以
編碼器輸出向量以創建加權組合。結果
（在代碼中稱為``attn_applied``）應該包含有關的信息
那個輸入序列的特定部分，從而有助於解碼器
選擇正確的輸出詞。


^^^^^^^^^^^^^^^^^

使用另一個前饋來計算注意力
layer``attn``，使用解碼器的輸入和隱藏狀態作為輸入。
因為訓練數據中有各種大小的句子，所以
實際創建和訓練這一層我們必須選擇一個最大值
它可以應用的句子長度（輸入長度，編碼器輸出）
至。最大長度的句子將使用所有註意力量，
而較短的句子只會使用前幾個。




In [215]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0)) #透過unsqueeze整理一下矩陣
        #bmm = batch matrix multiply 矩陣乘法

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
#Dropout防止Co-adaptation問題
#特征Co-adaptation指的是某个特征只有在另个特征出现时才有用。防止過度擬合，怕同一組的神經元都一直同一組導致過度擬合
#防止特征 Co-adaptation的方法是Dropout
#Dropout顾名思义就是在训练模型的时候随机扔掉一部分的输入 （即随机把一部分输入变成0）


<div class="alert alert-info"><h4>Note</h4><p>There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>`__.</p></div>

Training
========

Preparing Training Data
-----------------------

為了訓練，對於每一對我們將需要一個輸入張量（指數）
輸入句子中的單詞）和目標張量（單詞的索引）
目標句子）。在創建這些向量時，我們將附加
兩個序列的EOS令牌。



In [216]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token) #對序列加上結束的標記
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor) #回傳輸入張量 與 目標張量

Training the Model
------------------

為了訓練我們通過編碼器運行輸入句子，並保持跟踪
每個輸出和最新的隱藏狀態。然後給出解碼器
``<SOS>``標記作為它的第一個輸入，以及最後一個隱藏狀態
編碼器作為其第一個隱藏狀態。

“教師強迫(teacher_forcing)”是使用真實目標輸出的概念
每個下一個輸入，而不是使用解碼器的猜測作為下一個輸入。
使用教師強制使其收斂得更快，但是當受過訓練時
網絡被利用，它可能表現出來
不穩定<http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf>`__。

您可以觀察讀取的教師強製網絡的輸出
連貫的語法，但遠離正確的翻譯 - 
直覺上它已經學會了表示輸出語法並且可以“選擇”
一旦老師告訴它前幾個字，它就是“意思”
沒有正確地學習如何從翻譯中創建句子
首先。

由於PyTorch的autograd給我們的自由，我們可以隨機
選擇使用教師強製或不使用簡單的if語句。轉
``teacher_forcing_ratio``最多可以使用它。




In [217]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad() #初始化
    decoder_optimizer.zero_grad() #初始化

    input_length = input_tensor.size(0) 
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0 

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing: #透過真實的值，作為下一次的輸入
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input 
            #detach返回一個Variable結果

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

這是打印時間和估計時間的輔助函數
給出當前時間和進度％。



In [218]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


整個培訓過程如下：

 - 啟動計時器
 - 初始化優化程序和標準
 - 創建一組訓練對
 - 啟動空損陣列進行繪圖

然後我們多次調用``train``並偶爾打印進度（％）
例子，到目前為止的時間，估計的時間）和平均損失。

In [230]:
#def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
 #   start = time.time()
    #plot_losses = []
 #   print_loss_total = 0  # Reset every print_every
    #plot_loss_total = 0  # Reset every plot_every

 #   encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate) #選用SGD做優化
 #   decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
 #   training_pairs = [tensorsFromPair(random.choice(pairs))
 #                     for i in range(n_iters)] #隨機產生訓練資料組合
 #   criterion = nn.NLLLoss() #用NULLoss做評價
 
  #  for iter in range(1, n_iters):
  #      training_pair = training_pairs[iter]
  #      input_tensor = training_pair[0] #左邊第一個句子
  #      target_tensor = training_pair[1] #右邊第一個句子

  #      loss = train(input_tensor, target_tensor, encoder,
  #                   decoder, encoder_optimizer, decoder_optimizer, criterion)
  #      print_loss_total += loss
        #plot_loss_total += loss

  #      if iter % print_every == 0:
  #          print_loss_avg = print_loss_total / print_every
  #          print_loss_total = 0
  #          print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

   #     if iter % plot_every == 0:
   #         plot_loss_avg = plot_loss_total / plot_every
   #         plot_losses.append(plot_loss_avg)
   #         plot_loss_total = 0

    #showPlot(plot_losses)

In [237]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate) #選用SGD做優化
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)] #隨機產生訓練資料組合
    criterion = nn.NLLLoss() #用NULLoss做評價
 
    for iter in range(1, n_iters):
        training_pair = training_pairs[iter]
        input_tensor = training_pair[0] #左邊第一個句子
        target_tensor = training_pair[1] #右邊第一個句子

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss

        if iter % print_every == 0: #印出每一次的迭代過程
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

Plotting results
----------------

使用matplotlib繪製繪圖，使用損失值數組
訓練時保存了``plot_losses``。




In [221]:
#import matplotlib.pyplot as plt
#plt.switch_backend('agg')
#import matplotlib.ticker as ticker
#import numpy as np


#def showPlot(points):
#    plt.figure()
#    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
#    loc = ticker.MultipleLocator(base=0.2)
#    ax.yaxis.set_major_locator(loc)
#    plt.plot(points)

Evaluation （預測）
==========

評估與培訓大致相同，但沒有目標
我們只需將解碼器的預測反饋給每一步。
每當它預測一個單詞時，我們將它添加到輸出字符串，如果它
預測我們停在那裡的EOS令牌。我們還存儲了解碼器
注意輸出以便稍後顯示。




In [238]:
#evaluate與trainIters大致相同，插在於預測值的反饋
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad(): #no_grad上下文切斷
        input_tensor = tensorFromSentence(input_lang, sentence) 
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden() #初始化隱藏層

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device) #建立為空的矩陣

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length) #用最大的長度作為decoder_attentions的張量

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1) #一樣挑選有價值的index出來
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]


我們可以從訓練集中評估隨機句子並打印出來
輸入，目標​​和輸出做出一些主觀質量判斷：

In [239]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0]) #預測結果
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Training and Evaluating
=======================

有了所有這些輔助功能（它看起來像額外的工作，但是
它可以更容易地進行多個實驗）我們實際上可以
初始化網絡並開始培訓。

請記住輸入句子被嚴格過濾。對於這個小
數據集我們可以使用256個隱藏節點和一個相對較小的網絡
單個GRU層。在MacBook CPU上大約40分鐘後，我們會得到一些
合理的結果。

.. 注意：：
   如果你運行這個筆記本，你可以訓練，打斷內核，
   評估，並繼續培訓。註釋掉那裡的線條
   編碼器和解碼器被初始化並再次運行``trainIters``。




In [240]:
hidden_size = 1 #建議使用256個隱藏層
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device) #to(device)有GPU就跑
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
#dropout_p=0.1 轉為0的機率

trainIters(encoder1, attn_decoder1, 100, print_every=10) #迭代次數75000, 顯示次數5000

0m 0s (- 0m 1s) (10 10%) 9.5960
0m 0s (- 0m 1s) (20 20%) 9.5335
0m 0s (- 0m 1s) (30 30%) 9.4777
0m 0s (- 0m 0s) (40 40%) 9.4742
0m 0s (- 0m 0s) (50 50%) 9.3710
0m 0s (- 0m 0s) (60 60%) 9.3750
0m 1s (- 0m 0s) (70 70%) 9.2823
0m 1s (- 0m 0s) (80 80%) 9.3993
0m 1s (- 0m 0s) (90 90%) 9.2941


In [241]:
evaluateRandomly(encoder1, attn_decoder1)

> mon opinion est qu il a raison .
= in my opinion he s right .
< talks talks talks talks talks talks talks talks talks talks

> il n est plus celui qu il etait .
= he is not what he used to be .
< talks talks talks talks talks talks talks talks talks talks

> je suis fier de mon pere .
= i m proud of my father .
< talks talks talks talks talks talks talks talks talks talks

> je peux t aider .
= i can help you .
< talks talks talks talks talks talks talks talks talks talks

> je pense que nous sommes voisins .
= i think we re neighbors .
< talks talks talks talks talks talks talks talks talks talks

> de la fumee noire sortit de la cheminee .
= black smoke came out of the chimney .
< talks talks talks talks talks talks talks talks talks talks

> ces appareils photo sont fabriques au japon .
= these cameras are made in japan .
< talks talks talks talks talks talks talks talks talks talks

> que t as offert tom pour noel ?
= what did tom give you for christmas ?
< talks talks talks talk

Visualizing Attention
---------------------
注意機制的一個有用特性是其高度可解釋性
輸出。因為它用於加權特定的編碼器輸出
在輸入序列中，我們可以想像查找網絡最關注的位置
在每個時間步。

您可以簡單地運行``plt.matshow（attentions）``來查看注意輸出
顯示為矩陣，列為輸入步驟和行
輸出步驟：




In [227]:
#output_words, attentions = evaluate(
#    encoder1, attn_decoder1, "nous en avons presque fini")
#plt.matshow(attentions.numpy())
#plt.show()

為了獲得更好的觀看體驗，我們將完成添加軸的額外工作
和標籤：

In [247]:
#def showAttention(input_sentence, output_words, attentions):
    #Set up figure with colorbar
#    fig = plt.figure()
#    ax = fig.add_subplot(111)
#    cax = ax.matshow(attentions.numpy(), cmap='bone')
#    fig.colorbar(cax)

    # Set up axes
#    ax.set_xticklabels([''] + input_sentence.split(' ') +
#                     ['<EOS>'], rotation=90)
#    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
#    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
#    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

#    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    #showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("ces appareils photo sont fabriques au japon")

#evaluateAndShowAttention("elle est trop petit .")

#evaluateAndShowAttention("je ne crains pas de mourir .")

#evaluateAndShowAttention("c est un jeune directeur plein de talent .")

input = ces appareils photo sont fabriques au japon
output = talks talks talks talks talks talks talks talks talks talks
